In [7]:
from scipy.spatial.distance import cosine
from mtcnn.mtcnn import MTCNN
from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder, Normalizer
from sklearn.svm import SVC
from facenet_pytorch import MTCNN
import torch
import cv2
from numpy import asarray, savez_compressed, load, expand_dims, save,array
from PIL import Image, ImageDraw
import albumentations as A
import os 
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [8]:
def extract_face(filename, required_size=(224,224)):
    # load image from file
    image = cv2.imread(filename)
    # convert to RGB, if needed
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # convert to array
    pixels = asarray(image)
    # create the detector using default weights
    detector = mtcnn.MTCNN()
    # detect faces in the image
    results = detector.detect_faces(pixels)
    face_array = []
    # extract the bounding box from the first face
    for res in results:
        x1, y1, width, height = res['box']
        # confidence
        confidence = res['confidence']
        # bug fix
        x1, y1 = abs(x1), abs(y1)
        x2, y2 = x1+width, y1+height
        # greater than 0.95
        if confidence > 0.85:
            # extract the face
            face = pixels[y1:y2, x1:x2]
            image = Image.fromarray(face)
            image = image.resize(required_size)
            face_array.append(asarray(image))
    return face_array

In [11]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

Running on device: cuda:0


In [12]:
mtcn = MTCNN(margin=14,factor=0.6,keep_all=True,device=device)

In [13]:
def extract_fast_face(filename, required_size=(224,224)):
    img = cv2.imread(filename)
    img_rgb = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    pixels = asarray(img_rgb)
    boxes, _ = mtcn.detect(pixels)
    face_array = []
    for c, i in enumerate(_):
        if i > 0.95:
            box = boxes[c]
            f = img_rgb.crop((box[0], box[1], box[2], box[3]))
            image = f.resize(required_size)
            face_array.append(asarray(image))
    return face_array

In [14]:
def load_faces(directory):
    faces = list()
    #enumerate files
    for filename in os.listdir(directory):
        # path
        path = os.path.join(directory, filename)
        # get face
        face = extract_fast_face(path)
        # append all faces in an image into new faces array
        for pixel in face:
            faces.append(pixel)
    return faces

In [15]:
transform = A.Compose([
    A.Blur(blur_limit=3),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
#     A.OneOf([
#             A.MotionBlur(p=0.2),
#             A.MedianBlur(blur_limit=3, p=0.1),
#             A.Blur(blur_limit=3, p=0.1),
#         ], p=0.2),
#     A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=45, p=0.2),
])

In [16]:
# load a dataset that contains one subdir  for each class that in turn contains images
def load_dataset(dir_):
    X, y = list(), list()
    # enumerate folders, on per class
    for subdir in os.listdir(dir_):
        # path
        path = os.path.join(dir_, subdir)
        # skip any files that might be in the dir
        if not os.path.isdir(path):
            continue
        # load all faces in the subdirectory
#         if subdir != 'Atif Tahir':
#             continue
        faces = load_faces(path)
        new_faces = faces[:]  
        #augmentation
        if len(new_faces) < 2:
            k = 90
        elif len(new_faces) < 7:
            k = 30
        else:
            k = 2
        for face in faces:
            for i in range(k):
                transformed = transform(image=face)
                new_faces.append(transformed['image'])
        # create labels 
        labels = [subdir for _ in range(len(new_faces))]
        # summarize progress
        print(f"loaded {len(new_faces)} examples for class {subdir}")
        # store
        X.extend(new_faces)
        y.extend(labels)
    return asarray(X), asarray(y)

In [9]:
# load train dataset
trainX, trainy = load_dataset("data/train/")
print(trainX.shape, trainy.shape)

loaded 91 examples for class Aashir
loaded 91 examples for class Abeer
loaded 91 examples for class Anam Qureshi
loaded 91 examples for class Anaum Hamid
loaded 91 examples for class Aqsa Zahid
loaded 91 examples for class Atiya
loaded 91 examples for class Bakhtawer
loaded 91 examples for class Basit Ali
loaded 91 examples for class Danish
loaded 91 examples for class Dr. Aqsa Aslam
loaded 186 examples for class Dr. Atif
loaded 91 examples for class Dr. Fahad Sherwani
loaded 91 examples for class Dr. Farooque
loaded 91 examples for class Dr. Farrukh Hassan
loaded 91 examples for class Dr. Farrukh Salim
loaded 91 examples for class Dr. Farrukh Shahid
loaded 91 examples for class Dr. Ghufran
loaded 91 examples for class Dr. Jawwad
loaded 91 examples for class Dr. Nadeem Kafi
loaded 91 examples for class Dr. Nausheen
loaded 91 examples for class Dr. Nouman
loaded 91 examples for class Dr. Usama
loaded 91 examples for class Dr. Waheed
loaded 91 examples for class Dr. Zulfiqar
loaded 91 ex

In [13]:
testX, testy = load_dataset("data/val")

loaded 1 examples for class Aashir
loaded 5 examples for class Abdul Rehman
loaded 1 examples for class Abeer
loaded 11 examples for class ahmed
loaded 7 examples for class Ahmed Naeem
loaded 1 examples for class Anam Qureshi
loaded 1 examples for class Anaum Hamid
loaded 1 examples for class Aqsa Zahid
loaded 1 examples for class Atiya
loaded 1 examples for class Bakhtawer
loaded 1 examples for class Basit Ali
loaded 1 examples for class Danish
loaded 1 examples for class Dr. Aqsa Aslam
loaded 2 examples for class Dr. Atif
loaded 1 examples for class Dr. Fahad Sherwani
loaded 1 examples for class Dr. Farooque
loaded 1 examples for class Dr. Farrukh Hassan
loaded 1 examples for class Dr. Farrukh Salim
loaded 1 examples for class Dr. Farrukh Shahid
loaded 1 examples for class Dr. Ghufran
loaded 1 examples for class Dr. Jawwad
loaded 1 examples for class Dr. Nadeem Kafi
loaded 1 examples for class Dr. Nausheen
loaded 1 examples for class Dr. Nouman
loaded 1 examples for class Dr. Usama
l

In [10]:
savez_compressed('Backend/VGGFaceModelData/data.npz', trainX, trainy)

In [17]:
# extract faces and calculate face embeddings for a list of photo files
def get_embeddings(model,faces):
    # convert into an array of samples
    samples = faces.astype('float32')
    samples = expand_dims(samples, axis=0)
    # prepare the face for the model, e.g. center pixels
    samples = preprocess_input(samples, version=2)
    # perform prediction
    yhat = model.predict(samples)
    return yhat[0]

In [12]:
# load the face dataset
data = load('Backend/VGGFaceModelData/data.npz')
trainX, trainy = data['arr_0'], data['arr_1']
# trainX, trainy = data['arr_0'], data['arr_1']
print(f"Loaded: {trainX.shape, trainy.shape}")
# print(f"Loaded: {trainX.shape, trainy.shape}")
# load the facenet model
model = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3), pooling='avg')
# convert each face in the train set to an embedding
newTrainX = list()
for face_pixels in trainX:
    embedding = get_embeddings(model,face_pixels)
    newTrainX.append(embedding)
newTrainX = asarray(newTrainX)
print(newTrainX.shape)
# convert each face in the test set to an embedding
# newTestX = list()
# for face_pixels in testX:
#     embedding = get_embeddings(model,face_pixels)
#     newTestX.append(embedding)
# newTestX = asarray(newTestX)
# print(newTestX.shape)

Loaded: ((4190, 224, 224, 3), (4190,))







(4190, 2048)


In [13]:
savez_compressed('Backend/VGGFaceModelData/faceEmbeddings_1.npz', newTrainX, trainy)

In [18]:
data = load('Backend/VGGFaceModelData/faceEmbeddings.npz')
trainX, trainy, testX, testy = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']
print(f"Dataset: train={trainX.shape[0], testX.shape[0]}")
#normalize input vectors
in_encoder = Normalizer(norm='l2')
trainX = in_encoder.transform(trainX)
testX = in_encoder.transform(testX)
# label encode targets
out_encoder = LabelEncoder()
out_encoder.fit(trainy)
trainy = out_encoder.transform(trainy)
testy = out_encoder.transform(testy)
save('Backend/VGGFaceModelData/classes.npy', out_encoder.classes_)
# fit model
model = SVC(kernel='linear', probability=True)
model.fit(trainX, trainy)
# predict
yhat_train = model.predict(trainX)
yhat_test = model.predict(testX)
# score
score_train = accuracy_score(trainy, yhat_train)
score_test = accuracy_score(testy, yhat_test)
# summarize
print(f"Accuracy: train={score_train*100}, test={score_test*100}")

Dataset: train=(4450, 92)
Accuracy: train=100.0, test=98.91304347826086


In [10]:
import pickle
filename = 'finalized_new_model.sav'
pickle.dump(model, open(f"Backend/VGGFaceModelData/{filename}", 'wb'))

NameError: name 'model' is not defined

In [ ]:
import pickle
filename = 'finalized_new_model.sav'
model1 = pickle.load(open(f"Backend/VGGFaceModelData/{filename}", 'rb'))
encoder = LabelEncoder()
encoder.classes_ = load('Backend/VGGFaceModelData/classes.npy')
model = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3), pooling='avg')

In [19]:
def run(filename, required_size=(224,224)):
    img = cv2.imread(filename)
    img_rgb = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    boxes, _ = mtcn.detect(img_rgb)
    name = None
    conf = 0.95
    prob_= 50.00
    for c, i in enumerate(_):
        data = list()
        if i > 0.95:
            box = boxes[c]
            f = img_rgb.crop((box[0], box[1], box[2], box[3]))
            image = asarray(f.resize(required_size))
            data.append(get_embeddings(model, image))
            data = asarray(data)
            in_encoder = Normalizer(norm='l2')
            dataX = in_encoder.transform(data)
            class_ = model1.predict(dataX)
            yhat_prob = model1.predict_proba(dataX)
            predict_names = encoder.inverse_transform(class_)
            prob = yhat_prob[0, class_]*100
            if prob[0] > prob_:
                print(prob[0], predict_names[0])
                name = predict_names
                cv2.putText(img, f'{name[0]}', (int(box[0]), int(box[1])), cv2.FONT_ITALIC, 1, (0, 0, 255), 1)
    cv2.imshow('faces', img)
    cv2.waitKey(0) 

In [ ]:
run("testing_data/friends.jpeg")

50.79217058275164 Fouzan
55.631202558490855 Ahmed Naeem
55.62064912664858 ahmed


In [ ]:
import cv2
import math
import imageio
capture = cv2.VideoCapture("testing_data/")
frames_tracked = []

while True:
    IsTrue, frames = capture.read()
    if frames is None:
        break
    img_cv2 = asarray(frames)
    frames = Image.fromarray(cv2.cvtColor(frames, cv2.COLOR_BGR2RGB))
    boxes, _ = mtcn.detect(frames)
    required_size=(224, 224)
    name = None
    conf = 0.90
    prob_= 50.00
    if _[0] is not None:
        for c, i in enumerate(_):
            data = list()
            if i > conf:
                box = boxes[c]
                f = frames.crop((box[0], box[1], box[2], box[3]))
                image = asarray(f.resize(required_size))
                data.append(get_embeddings(model, image))
                data = asarray(data)
                in_encoder = Normalizer(norm='l2')
                dataX = in_encoder.transform(data)
                class_ = model1.predict(dataX)
                yhat_prob = model1.predict_proba(dataX)
                predict_names = encoder.inverse_transform(class_)
                prob = yhat_prob[0, class_]*100
                if prob[0] > prob_:
                    name = predict_names
                    x, y, w, h = math.floor(box[0]), math.floor(box[1]), abs(math.floor(box[0])-math.floor(box[2])), abs(math.floor(box[1])-math.floor(box[3]))
                    cv2.putText(img_cv2, f'{name[0]}', (int(box[0]), int(box[1])), cv2.FONT_ITALIC, 1, (0, 0, 255), 1)
                    frame_draw = frames.copy()
                    draw = ImageDraw.Draw(frame_draw)
                    draw.text((box[0], box[1]), "ahmed", fill ="blue",align ="right") 
                    draw.rectangle(box.tolist(), outline=(255, 0, 0), width=1)
                    frames_tracked.append(frame_draw.resize((640, 360), Image.BILINEAR))
    cv2.imshow('friends', img_cv2)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
#     person_name, box_dimensions = face_recognize(frames)
#     x,y,w,h = box_dimensions
#     cv2.rectangle(frames,(x,y),(x+w,y+h),(0,255,0),2)
#     cv2.putText(frames,person_name,(x+w+10,y+h),0,0.3,(0,255,0))
#     image_lst.append(frames)

capture.release()
cv2.destroyAllWindows()

# # Convert to gif using the imageio.mimsave method
# imageio.mimsave('ben_afflek_processed.gif', image_lst, fps=10)

In [ ]:
# determine if a candidate face is a match for a known face
def is_match(known_embedding, candidate_embedding, thresh=0.5):
    # calculate distance between embeddings
    score = cosine(known_embedding, candidate_embedding)
    if score <= thresh:
        print('>face is a Match (%.3f <= %.3f)' % (score, thresh))
    else:
        print('>face is NOT a Match (%.3f > %.3f)' % (score, thresh))
 
# define filenames
filenames = ['testing_data/ahmed_new.jpg', 'testing_data/ahmed1.1.jpeg', 'testing_data/fouz.jpeg', 'testing_data/fouzan.jpeg','testing_data/ahmed1.2.jpeg']
# get embeddings file filenames
embeddings = get_embeddings(filenames)
print(embeddings[0].shape)
print(embeddings[1].shape)
# # define sharon stone
# sharon_id = embeddings[0]
# # verify known photos of sharon
print('Positive Tests')
is_match(embeddings[0], embeddings[4])
# is_match(embeddings[0], embeddings[2])em
# # verify known photos of other people
print('Negative Tests')
is_match(embeddings[0], embeddings[2])

In [18]:
import numpy as np
def distance(emb1, emb2):
    return np.sum(np.square(emb1 - emb2))

In [19]:
def findCosineDistance(source_representation, test_representation):
    a = np.matmul(np.transpose(source_representation), test_representation)
    b = np.sum(np.multiply(source_representation, source_representation))
    c = np.sum(np.multiply(test_representation, test_representation))
    return 1 - (a / (np.sqrt(b) * np.sqrt(c)))
 
def findEuclideanDistance(source_representation, test_representation):
    euclidean_distance = source_representation - test_representation
    euclidean_distance = np.sum(np.multiply(euclidean_distance, euclidean_distance))
    euclidean_distance = np.sqrt(euclidean_distance)
    return euclidean_distance

In [18]:
data = load('Backend/VGGFaceModelData/faceEmbeddings.npz')
trainX, trainy = data['arr_0'], data['arr_1']
model = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3), pooling='avg')
import cv2
import math
import imageio
capture = cv2.VideoCapture("")
frames_tracked = []
diff = 0
cosine_similarity = None
new_trainX = list()
for i in range(0, len(trainy)):
    if trainy[i] == 'Javeria Iftikhar':
        new_trainX.append(trainX[i])

new_trainX = asarray(new_trainX)
name = "Javeria Iftikhar"
while True:
    IsTrue, frames = capture.read()
    if frames is None:
        break
#     frames = cv2.rotate(frames, cv2.ROTATE_180)
    img_cv2 = asarray(frames)
    frames = Image.fromarray(cv2.cvtColor(frames, cv2.COLOR_BGR2RGB))
    boxes, _ = mtcn.detect(frames)
    required_size=(224,224)
    conf = 0.50
    prob_= 70.00
    epsilon = 0.40
    if _[0] is not None:
        for c, i in enumerate(_):
            if i > 0.70:
                box = boxes[c]
                f = frames.crop((box[0], box[1], box[2], box[3]))
                image = f.resize(required_size)
                data = get_embeddings(model, asarray(image))
                for i, d in enumerate(trainX):
                    diff = distance(data,d)
                    name = trainy[i]
                    cosine_similarity = findCosineDistance(data, d)
                    x, y, w, h = math.floor(box[0]), math.floor(box[1]), abs(math.floor(box[0])-math.floor(box[2])), abs(math.floor(box[1])-math.floor(box[3]))
                    cv2.rectangle(img_cv2, (x, y), (x+w, y+h), (255, 0, 0), thickness=1)
                    
                    if cosine_similarity <= epsilon:
                        print(cosine_similarity, name)
                        cv2.putText(img_cv2, f'{name}-{round(cosine_similarity,3)}', (int(box[0]), int(box[1])), cv2.FONT_ITALIC, 1, (0, 0, 255), 1)
    cv2.imshow('friends', img_cv2)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
capture.release()
cv2.destroyAllWindows()



0.3954737186431885 Javeria Iftikhar
0.3875473141670227 Javeria Iftikhar
0.39362239837646484 Javeria Iftikhar
0.39294642210006714 Javeria Iftikhar
0.386577308177948 Javeria Iftikhar
0.3939790725708008 Javeria Iftikhar
0.39294642210006714 Javeria Iftikhar
0.39294642210006714 Javeria Iftikhar
0.3896399140357971 Javeria Iftikhar
0.3939790725708008 Javeria Iftikhar
0.39215368032455444 Javeria Iftikhar
0.39294642210006714 Javeria Iftikhar
0.3896399140357971 Javeria Iftikhar
0.3915807008743286 Javeria Iftikhar
0.39799922704696655 Javeria Iftikhar
0.38186126947402954 Javeria Iftikhar
0.3896399140357971 Javeria Iftikhar
0.3896399140357971 Javeria Iftikhar
0.3896399140357971 Javeria Iftikhar
0.3873748779296875 Javeria Iftikhar
0.3866862654685974 Javeria Iftikhar
0.3896399140357971 Javeria Iftikhar
0.3939790725708008 Javeria Iftikhar
0.39294642210006714 Javeria Iftikhar
0.39294642210006714 Javeria Iftikhar
0.3948379158973694 Javeria Iftikhar
0.3896399140357971 Javeria Iftikhar
0.39294642210006714

In [ ]:
data = load('Backend/VGGFaceModelData/faceEmbeddings.npz')
trainX, trainy, testX, testy = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']
model = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3), pooling='avg')
import cv2
import math
import imageio
capture = cv2.VideoCapture("testing_data/R12NewVideo.mp4")
frames = capture.get(cv2.CAP_PROP_FRAME_COUNT)
frame_index = frames-1
diff = 0
cosine_similarity = None

while(frame_index!=0):
    capture.set(cv2.CAP_PROP_POS_FRAMES, frame_index)
    IsTrue, frames = capture.read()
    if frames is None:
        break
#     frames = cv2.rotate(frames, cv2.ROTATE_180)
    img_cv2 = asarray(frames)
    frames = Image.fromarray(cv2.cvtColor(frames, cv2.COLOR_BGR2RGB))
    boxes, _ = mtcn.detect(frames)
    required_size=(224,224)
    name = None
    conf = 0.50
    prob_= 70.00
    epsilon = 0.34
    frame_index = frame_index-1
    if _[0] is not None:
        for c, i in enumerate(_):
            if i > 0.70:
                box = boxes[c]
                f = frames.crop((box[0], box[1], box[2], box[3]))
                image = f.resize(required_size)
                data = get_embeddings(model, asarray(image))
                for i, d in enumerate(trainX):
                    diff = distance(data,d)
                    cosine_similarity = findCosineDistance(data, d)
                    name = trainy[i]
                    x, y, w, h = math.floor(box[0]), math.floor(box[1]), abs(math.floor(box[0])-math.floor(box[2])), abs(math.floor(box[1])-math.floor(box[3]))
                    cv2.rectangle(img_cv2, (x, y), (x+w, y+h), (255, 0, 0), thickness=1)
                    if cosine_similarity <= epsilon:
                        print(cosine_similarity, name)
                        cv2.putText(img_cv2, f'{name}', (int(box[0]), int(box[1])), cv2.FONT_ITALIC, 1, (0, 0, 255), 1)
    cv2.imshow('friends', img_cv2)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
capture.release()
cv2.destroyAllWindows()



## Get Total Frames

In [20]:
from moviepy.editor import *
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
import os

def get_duration(filename):
	clip = VideoFileClip(filename)
	return clip.duration, clip.fps

In [28]:
path = "testing_data/DanishKhan.mp4"

In [29]:
duration, fps = get_duration(path)

In [30]:
total_frames = duration * fps

In [31]:
total_frames

70654.0

In [32]:
import cv2
import math
import imageio

data = load('VGGFaceModelData/faceEmbeddings_1.npz')
trainX, trainy = data['arr_0'], data['arr_1']
model = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3), pooling='avg')

capture = cv2.VideoCapture("testing_data/DanishKhan.mp4")
frames = 0
frame_index = 1
diff = 0
cosine_similarity = None

while(frame_index<total_frames):
    capture.set(cv2.CAP_PROP_POS_FRAMES, frame_index)
    IsTrue, frames = capture.read()
    print(frame_index)
    if frames is None:
        break
#     frames = cv2.rotate(frames, cv2.ROTATE_180)
    img_cv2 = asarray(frames)
    frames = Image.fromarray(cv2.cvtColor(frames, cv2.COLOR_BGR2RGB))
    boxes, _ = mtcn.detect(frames)
    required_size=(224,224)
    name = None
    conf = 0.50
    prob_= 70.00
    epsilon = 0.34
    frame_index = frame_index + 8
    if _[0] is not None:
        for c, i in enumerate(_):
            if i > 0.90:
                box = boxes[c]
                f = frames.crop((box[0], box[1], box[2], box[3]))
                image = f.resize(required_size)
                data = get_embeddings(model, asarray(image))
                for i, d in enumerate(trainX):
                    diff = distance(data,d)
                    cosine_similarity = findCosineDistance(data, d)
                    name = trainy[i]
                    x, y, w, h = math.floor(box[0]), math.floor(box[1]), abs(math.floor(box[0])-math.floor(box[2])), abs(math.floor(box[1])-math.floor(box[3]))
                    cv2.rectangle(img_cv2, (x, y), (x+w, y+h), (255, 0, 0), thickness=1)
                    if cosine_similarity <= epsilon:
                        print(cosine_similarity, name)
                        cv2.putText(img_cv2, f'{name}', (int(box[0]), int(box[1])), cv2.FONT_ITALIC, 1, (0, 0, 255), 1)
    cv2.imshow('friends', img_cv2)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
capture.release()
cv2.destroyAllWindows()


1
9
17
25
33
41
49
57
65
73
81
89
97
105
113
121
129
137
145
153
161
169
177
185
193
201
209
217
225
233
241
249
257
265
273
281
289
297
305
313
321
329
337
345
353
361
369
377
385
393
401
409
417
425
433
441
449
457
465
473
481
489
497
505
513
521
529
537
545
553
561
569
577
585
593
601
609
617
625
633
641
649
657
665
673
681
689
697
705
713
721
729
737
745
753
761
769
777
785
793
801
809
817
825
833
841
849
857
865
873
881
889
897
905
913
921
929
937
945
953
961
969
977
985
993
1001
1009
1017
1025
1033
1041
1049
1057
1065
1073
1081
1089
1097
1105
1113
1121
1129
1137
1145
1153
1161
1169
1177
1185
1193
1201
1209
1217
1225
1233
1241
1249
1257
1265
1273
1281
1289
1297
1305
1313
1321
1329
1337
1345
1353
1361
1369
1377
1385
1393
1401
1409
1417
1425
1433
1441
1449
1457
1465
1473
1481
1489
1497
1505
1513
1521
1529
1537
1545
1553
1561
1569
1577
1585
1593
1601
1609
1617
1625
1633
1641
1649
1657
1665
1673
1681
1689
1697
1705
1713
1721
1729
1737
1745
1753
1761
1769
1777
1785
1793
1801
1809
1817


KeyboardInterrupt: 

In [33]:
data = load('VGGFaceModelData/faceEmbeddings_1.npz')
trainX, trainy = data['arr_0'], data['arr_1']
trainy = np.unique(trainy)
trainy

array(['Aashir', 'Abeer', 'Anam Qureshi', 'Anaum Hamid', 'Aqsa Zahid',
       'Atiya', 'Bakhtawer', 'Basit Ali', 'Danish', 'Dr. Aqsa Aslam',
       'Dr. Atif', 'Dr. Fahad Sherwani', 'Dr. Farooque',
       'Dr. Farrukh Hassan', 'Dr. Farrukh Salim', 'Dr. Farrukh Shahid',
       'Dr. Ghufran', 'Dr. Jawwad', 'Dr. Nadeem Kafi', 'Dr. Nausheen',
       'Dr. Nouman', 'Dr. Rafi', 'Dr. Usama', 'Dr. Waheed',
       'Dr. Zulfiqar', 'Erum', 'Fizza', 'Hajra', 'Hamza Ahmed', 'Hassan',
       'Jamil', 'Javeria Iftikhar', 'Kashan', 'M. Shahzad', 'Mafaza',
       'Musawar', 'Nida Munawar', 'Rabia Tabassum', 'Romasha', 'Safia',
       'Shaharbano', 'Shahbaz', 'Shoaib Rauf', 'Sobia', 'Sohail', 'Zain',
       'Zumar'], dtype='<U18')

In [45]:
import cv2
import math
import imageio

data = load('VGGFaceModelData/faceEmbeddings_1.npz')
trainX, trainy = data['arr_0'], data['arr_1']
model = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3), pooling='avg')

capture = cv2.VideoCapture("testing_data/AtifTahir.mp4")
frames = 0
frame_index = 1
diff = 0
cosine_similarity = None
new_trainX = list()
for i in range(0, len(trainy)):
    if trainy[i] == 'Dr. Atif':
        new_trainX.append(trainX[i])

new_trainX = asarray(new_trainX)
name = "Dr. Atif"

while(frame_index<total_frames):
    capture.set(cv2.CAP_PROP_POS_FRAMES, frame_index)
    IsTrue, frames = capture.read()
    print(frame_index)
    if frames is None:
        break
#     frames = cv2.rotate(frames, cv2.ROTATE_180)
    img_cv2 = asarray(frames)
    frames = cv2.cvtColor(frames, cv2.COLOR_BGR2RGB)
    
    frames = Image.fromarray(frames)
    boxes, _ = mtcn.detect(frames)
    required_size=(224,224)
    name = None
    conf = 0.50
    prob_= 70.00
    epsilon = 0.45
    frame_index = frame_index + 16
    if _[0] is not None:
        for c, i in enumerate(_):
            if i > 0.70:
                box = boxes[c]
                f = frames.crop((box[0], box[1], box[2], box[3]))
                image = f.resize(required_size)
                data = get_embeddings(model, asarray(image))
                for i, d in enumerate(new_trainX):
                    name = "Dr. Atif"
                    cosine_similarity = findCosineDistance(data, d)
                    x, y, w, h = math.floor(box[0]), math.floor(box[1]), abs(math.floor(box[0])-math.floor(box[2])), abs(math.floor(box[1])-math.floor(box[3]))
                    cv2.rectangle(img_cv2, (x, y), (x+w, y+h), (255, 0, 0), thickness=1)
                    if cosine_similarity <= epsilon:
                        print(cosine_similarity, name)
                        cv2.putText(img_cv2, f'{name}', (int(box[0]), int(box[1])), cv2.FONT_ITALIC, 1, (0, 0, 255), 1)
    cv2.imshow('friends', img_cv2)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
capture.release()
cv2.destroyAllWindows()

1
17
33
0.3463847041130066 Dr. Atif
0.3784797787666321 Dr. Atif
0.4471306800842285 Dr. Atif
0.437344491481781 Dr. Atif
0.32960766553878784 Dr. Atif
0.3276466131210327 Dr. Atif
0.32960766553878784 Dr. Atif
0.3440698981285095 Dr. Atif
0.3463847041130066 Dr. Atif
0.34775543212890625 Dr. Atif
0.32960766553878784 Dr. Atif
0.3501056432723999 Dr. Atif
0.3440698981285095 Dr. Atif
0.3440698981285095 Dr. Atif
0.32960766553878784 Dr. Atif
0.3440698981285095 Dr. Atif
0.33057111501693726 Dr. Atif
0.3545231819152832 Dr. Atif
0.32960766553878784 Dr. Atif
0.32960766553878784 Dr. Atif
0.3463847041130066 Dr. Atif
0.3276466131210327 Dr. Atif
0.3276466131210327 Dr. Atif
0.3463847041130066 Dr. Atif
0.3463847041130066 Dr. Atif
0.32960766553878784 Dr. Atif
0.3440698981285095 Dr. Atif
0.32960766553878784 Dr. Atif
0.3440698981285095 Dr. Atif
0.32960766553878784 Dr. Atif
0.3440698981285095 Dr. Atif
0.3303700089454651 Dr. Atif
0.32960766553878784 Dr. Atif
0.3340054154396057 Dr. Atif
0.3784797787666321 Dr. Atif
0

321
337
353
0.4081938862800598 Dr. Atif
0.4081938862800598 Dr. Atif
0.40821176767349243 Dr. Atif
0.40441691875457764 Dr. Atif
0.43033552169799805 Dr. Atif
0.40441691875457764 Dr. Atif
0.4081938862800598 Dr. Atif
0.4081938862800598 Dr. Atif
0.40441691875457764 Dr. Atif
0.431532621383667 Dr. Atif
0.4319767951965332 Dr. Atif
0.40441691875457764 Dr. Atif
0.4081938862800598 Dr. Atif
0.4319767951965332 Dr. Atif
0.40441691875457764 Dr. Atif
0.40441691875457764 Dr. Atif
0.40441691875457764 Dr. Atif
0.4081938862800598 Dr. Atif
0.41053658723831177 Dr. Atif
0.43033552169799805 Dr. Atif
0.4319767951965332 Dr. Atif
0.4081938862800598 Dr. Atif
0.43033552169799805 Dr. Atif
0.43033552169799805 Dr. Atif
0.3851557970046997 Dr. Atif
0.43040984869003296 Dr. Atif
0.40441691875457764 Dr. Atif
0.4319767951965332 Dr. Atif
0.44549840688705444 Dr. Atif
0.40441691875457764 Dr. Atif
0.4081938862800598 Dr. Atif
0.43728840351104736 Dr. Atif
369
385
401
417
433
449
465
481
497
513
529
545
561
577
593
609
625
641
657

2065
0.44071775674819946 Dr. Atif
0.36900418996810913 Dr. Atif
0.37915700674057007 Dr. Atif
0.44004499912261963 Dr. Atif
0.44071775674819946 Dr. Atif
0.4421873688697815 Dr. Atif
0.44004499912261963 Dr. Atif
0.44004499912261963 Dr. Atif
0.44004499912261963 Dr. Atif
0.44071775674819946 Dr. Atif
0.44071775674819946 Dr. Atif
0.44071775674819946 Dr. Atif
0.44004499912261963 Dr. Atif
0.44004499912261963 Dr. Atif
0.44004499912261963 Dr. Atif
0.4387716054916382 Dr. Atif
0.43900513648986816 Dr. Atif
0.36900418996810913 Dr. Atif
0.3608730435371399 Dr. Atif
0.36991286277770996 Dr. Atif
0.3885957598686218 Dr. Atif
0.36991286277770996 Dr. Atif
0.36900418996810913 Dr. Atif
0.36900418996810913 Dr. Atif
0.36991286277770996 Dr. Atif
0.34666502475738525 Dr. Atif
0.38714754581451416 Dr. Atif
0.36991286277770996 Dr. Atif
0.36900418996810913 Dr. Atif
0.38714754581451416 Dr. Atif
0.36991286277770996 Dr. Atif
0.36991286277770996 Dr. Atif
0.36991286277770996 Dr. Atif
0.36900418996810913 Dr. Atif
0.41315549612

KeyboardInterrupt: 

In [27]:
import cv2
import math
import imageio

from random import randint


data = load('VGGFaceModelData/faceEmbeddings_1.npz')
trainX, trainy = data['arr_0'], data['arr_1']
model = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3), pooling='avg')

capture = cv2.VideoCapture("testing_data/mm.mp4")
frames = 0
frame_index = 1
diff = 0
cosine_similarity = None
new_trainX = list()
for i in range(0, len(trainy)):
    if trainy[i] == 'Dr. Rafi':
        new_trainX.append(trainX[i])

new_trainX = asarray(new_trainX)
name = "Dr. Rafi"
framess = []
faces = []
data1 = []
while(frame_index<int(total_frames)):
    capture.set(cv2.CAP_PROP_POS_FRAMES, frame_index)
    IsTrue, frames = capture.read()
    frame_index = frame_index + 8
    if len(framess) >= 60 or frames is None:
#         resize = 0.5
#         framess = [cv2.resize(f, (int(f.shape[1] * resize), int(f.shape[0] * resize))) for f in framess]

        boxes, _ = mtcn.detect(framess[::4])
        required_size=(224,224)
        name = None
        conf = 0.50
        prob_= 70.00
        epsilon = 0.45
        for i, frame in enumerate(framess):
            box_ind = int(i / 4)
            if boxes[box_ind] is None:
                continue
            for ind,box in enumerate(boxes[box_ind]):
                if _[box_ind][ind] > 0.90:
                    img_cv2 = asarray(frame)
                    frame = Image.fromarray(img_cv2)
                    f = frame.crop((box[0], box[1], box[2], box[3]))
                    faces.append(f)
#                     image = f.resize(required_size)
#                     data1.append(get_embeddings(model, asarray(image)))
#                     print(i)
#                     name = "Dr. Rafi"
#                     cosine_similarity = findCosineDistance(data, new_trainX[0])  
#                     if cosine_similarity <= epsilon:
#                         print(cosine_similarity, name)
        print(frame_index, len(faces))
        framess = []
        faces = []
    else:
        frames = cv2.cvtColor(frames, cv2.COLOR_BGR2RGB)
        framess.append(frames)

capture.release()









489 1124
977 1092
1465 1172
1953 1224
2441 1168
2665 504
